# Implémentation de ResNet à partir de zéro

Residual Network, ou ResNet en abrégé, constitue l'une des avancées les plus révolutionnaires en matière d'apprentissage en profondeur. Cette architecture repose sur un composant appelé module résiduel, qui nous permet d'ensemble des réseaux avec des profondeurs impensables il y a quelques années. Il existe des variantes de ResNet qui ont plus de 100 couches, sans aucune perte de performances !


Dans cette recette, nous allons implémenter ResNet à partir de zéro et le former sur le remplacement difficile de CIFAR-10, CINIC-10

Nous n'expliquerons pas ResNet en profondeur, c'est donc une bonne idée de vous familiariser avec l'architecture si vous êtes intéressé par les détails. Vous pouvez lire l'article original ici : https://arxiv.org/abs/1512.03385.

Suivez ces étapes pour mettre en œuvre ResNet à partir de zéro :
**1.** Importez tous les modules nécessaires :

In [ ]:
import os

import numpy as np
import tarfile
import tensorflow as tf
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.layers import *
from tensorflow.keras.models import *
from tensorflow.keras.regularizers import l2
from tensorflow.keras.utils import get_file

**2.** Dénissez un alias à l'option tf.data.expertimental.AUTOTUNE, que nous utiliserons plus tard :

In [ ]:
AUTOTUNE = tf.data.experimental.AUTOTUNE
TRAIN = True

**3.** Dénissez une fonction pour créer un module résiduel dans l'architecture ResNet. Commençons par spécifier la signature de la fonction et implémenter le premier bloc :

In [ ]:
def residual_module(data,
                    filters,
                    stride,
                    reduce=False,
                    reg=0.0001,
                    bn_eps=2e-5,
                    bn_momentum=0.9):
    # The shortcut branch of the ResNet module should be
    # initialized as the input (identity) data
    shortcut = data

    # The first block of the Resnet module are the 1x1 CONVs
    bn_1 = BatchNormalization(axis=-1,
                              epsilon=bn_eps,
                              momentum=bn_momentum)(data)
    act_1 = ReLU()(bn_1)
    conv_1 = Conv2D(filters=int(filters / 4.),
                    kernel_size=(1, 1),
                    use_bias=False,
                    kernel_regularizer=l2(reg))(act_1)

    # ResNet's module second block are 3x3 convolutions.
    bn_2 = BatchNormalization(axis=-1,
                              epsilon=bn_eps,
                              momentum=bn_momentum)(conv_1)
    act_2 = ReLU()(bn_2)
    conv_2 = Conv2D(filters=int(filters / 4.),
                    kernel_size=(3, 3),
                    strides=stride,
                    padding='same',
                    use_bias=False,
                    kernel_regularizer=l2(reg))(act_2)

    # The third block of the ResNet module is another set of
    # 1x1 convolutions.
    bn_3 = BatchNormalization(axis=-1,
                              epsilon=bn_eps,
                              momentum=bn_momentum)(conv_2)
    act_3 = ReLU()(bn_3)
    conv_3 = Conv2D(filters=filters,
                    kernel_size=(1, 1),
                    use_bias=False,
                    kernel_regularizer=l2(reg))(act_3)

    # If we are to reduce the spatial size, apply a 1x1
    # convolution to the shortcut
    if reduce:
        shortcut = Conv2D(filters=filters,
                          kernel_size=(1, 1),
                          strides=stride,
                          use_bias=False,
                          kernel_regularizer=l2(reg))(act_1)

    x = Add()([conv_3, shortcut])

    return x

**4.** Dénissez une fonction pour créer un réseau ResNet personnalisé :

In [ ]:
def build_resnet(input_shape,
                 classes,
                 stages,
                 filters,
                 reg=1e-3,
                 bn_eps=2e-5,
                 bn_momentum=0.9):
    inputs = Input(shape=input_shape)
    x = BatchNormalization(axis=-1,
                           epsilon=bn_eps,
                           momentum=bn_momentum)(inputs)

    x = Conv2D(filters[0], (3, 3),
               use_bias=False,
               padding='same',
               kernel_regularizer=l2(reg))(x)

    for i in range(len(stages)):
        # Initialize the stride, then apply a residual module
        # used to reduce the spatial size of the input volume.
        stride = (1, 1) if i == 0 else (2, 2)
        x = residual_module(data=x,
                            filters=filters[i + 1],
                            stride=stride,
                            reduce=True,
                            bn_eps=bn_eps,
                            bn_momentum=bn_momentum)

        # Loop over the number of layers in the stage.
        for j in range(stages[i] - 1):
            x = residual_module(data=x,
                                filters=filters[i + 1],
                                stride=(1, 1),
                                bn_eps=bn_eps,
                                bn_momentum=bn_momentum)

    x = BatchNormalization(axis=-1,
                           epsilon=bn_eps,
                           momentum=bn_momentum)(x)
    x = ReLU()(x)
    x = AveragePooling2D((8, 8))(x)

    x = Flatten()(x)
    x = Dense(classes, kernel_regularizer=l2(reg))(x)
    x = Softmax()(x)

    return Model(inputs, x, name='resnet')

**5.** Dénissez une fonction pour charger une image et ses étiquettes encodées à One_hot, en fonction de son chemin de chier :

In [ ]:
def load_image_and_label(image_path, target_size=(32, 32)):
    image = tf.io.read_file(image_path)
    image = tf.image.decode_png(image, channels=3)
    image = tf.image.convert_image_dtype(image, np.float32)
    image -= CINIC_MEAN_RGB  # Mean normalize
    image = tf.image.resize(image, target_size)

    label = tf.strings.split(image_path, os.path.sep)[-2]
    label = (label == CINIC_10_CLASSES)  # One-hot encode.
    label = tf.dtypes.cast(label, tf.float32)

    return image, label

**6.** Dénissez une fonction pour créer une instance tf.data.Dataset d'images et d'étiquettes à partir d'un modèle de type glob qui fait référence au dossier où se trouvent les images :

In [ ]:
def prepare_dataset(data_pattern, shuffle=False):
    dataset = (tf.data.Dataset
               .list_files(data_pattern)
               .map(load_image_and_label,
                    num_parallel_calls=AUTOTUNE)
               .batch(BATCH_SIZE))

    if shuffle:
        dataset = dataset.shuffle(BUFFER_SIZE)

    return dataset.prefetch(BATCH_SIZE)

**7.** Dénir les valeurs RGB moyennes du jeu de données CINIC-10, qui sont utilisées dans la fonction load_image_and_label() pour normaliser les images (cette information est disponible sur le site ofciel CINIC-10)

In [ ]:
CINIC_MEAN_RGB = np.array([0.47889522, 0.47227842, 0.43047404])

**8.** Déﬁnissez les classes du jeu de données CINIC-10 :

In [ ]:
CINIC_10_CLASSES = ['airplane', 'automobile', 'bird', 'cat',
                    'deer', 'dog', 'frog', 'horse', 'ship',
                    'truck']

**9.** Téléchargez et extrayez le jeu de données CINIC-10 dans le répertoire à votre choix

In [ ]:
DATASET_URL = ("https://datashare.is.ed.ac.uk/bitstream/handle/10283/3192/CINIC-10.tar.gz?sequence=4&isAllowed=y")

DATA_NAME = 'cinic10'
FILE_EXTENSION = 'tar.gz'
FILE_NAME = '.'.join([DATA_NAME, FILE_EXTENSION])

downloaded_file_location = get_file(origin=DATASET_URL,
                                    fname=FILE_NAME,
                                    extract=False)



687554560/687544992 [==============================] - 1713s 2us/step


In [ ]:
data_directory, _ = (downloaded_file_location
                     .rsplit(os.path.sep, maxsplit=1))
data_directory = os.path.sep.join([data_directory, DATA_NAME])
tar = tarfile.open(downloaded_file_location)
if not os.path.exists(data_directory):
  tar.extractall(data_directory)

**10.** Définissez les modèles de type glob pour les sous-ensembles d'entraînement, de test et de validation 

In [ ]:
train_pattern = os.path.sep.join(
    [data_directory, 'train/*/*.png'])
test_pattern = os.path.sep.join(
    [data_directory, 'test/*/*.png'])
valid_pattern = os.path.sep.join(
    [data_directory, 'valid/*/*.png'])

**11.** Préparez les ensembles de données :

In [ ]:
BATCH_SIZE = 128
BUFFER_SIZE = 1024
train_dataset = prepare_dataset(train_pattern, shuffle=True)
test_dataset = prepare_dataset(test_pattern)
valid_dataset = prepare_dataset(valid_pattern)

**12.** Construisez, compilez et entraînez un modèle ResNet. Comme il s'agit d'un processus qui prend du temps, nous enregistrerons une version du modèle après chaque époque, en utilisant le rappel ModelCheckpoint() :

In [ ]:
TRAIN = False
if TRAIN:
    model = build_resnet(input_shape=(32, 32, 3),
                         classes=10,
                         stages=(9, 9, 9),
                         filters=(64, 64, 128, 256),
                         reg=5e-3)
    model.compile(loss='categorical_crossentropy',
                  optimizer='rmsprop',
                  metrics=['accuracy'])

    model_checkpoint_callback = ModelCheckpoint(
        filepath='./model.{epoch:02d}-{val_accuracy:.2f}.hdf5',
        save_weights_only=False,
        monitor='val_accuracy')

    EPOCHS = 2
    model.fit(train_dataset,
              validation_data=valid_dataset,
              epochs=EPOCHS,
              callbacks=[model_checkpoint_callback])

Epoch 1/2


**13.** Chargez le meilleur modèle et évaluez-le sur l'ensemble de test 

In [ ]:
model = load_model('model.38-0.72.hdf5')
result = model.evaluate(test_dataset)
print(f'Test accuracy: {result[1]}')

La clé de ResNet est le module résiduel, que nous avons implémenté à l'étape 3. Un module résiduel est une micro-architecture qui peut être réutilisée plusieurs fois pour créer une macro-architecture, atteignant ainsi de grandes profondeurs. La fonction résiduelle_module() reçoit les données d'entrée (data), le nombre de filtres, le stride des blocs convolutifs, un indicateur de réduction pour indiquer si l'on veut réduire la taille spatiale du raccourci branche en appliquant une convolution 1x1 (une technique utilisée pour réduire la dimensionnalité des volumes de sortie des filtres), et des paramètres pour ajuster la quantité de régularisation (reg) et de normalisation par lots appliquée aux diférentes couches (bn_eps et bn_momentum) .

Un module résiduel comprend deux branches : la première est la connexion de saut(skip), également appelée branche de raccourci, qui est fondamentalement la même que l'entrée. La seconde ou branche principale est composée de trois blocs de convolution : un 1x1 avec un quart des filtres, un 3x3, également avec un quart des filtres, et enfin un autre 1x1, qui utilise tous les filtres. Le raccourci et les branches principales sont finalement concaténés à l'aide de la couche Add().

build_network() permet de spécifier le nombre d'étapes à utiliser, ainsi que le nombre de filter par étape. Nous commençons par appliquer une convolution 3x3 à l'entrée (après avoir été normalisée par lot). Ensuite, nous procédons à la création des étapes. Un étage est une série de modules résiduels connectés les uns aux autres. La longueur de la liste des étapes contrôle le nombre d'étapes à créer, et chaque élément de cette liste contrôle le nombre de couches dans cette étape particulière. Le paramètre de filtres contient le nombre de filtres à utiliser dans chaque bloc résiduel d'une étape. Enfin, nous avons construit un réseau entièrement connecté, activé par Sofmax, au-dessus des étages avec autant d'unités qu'il y a de classes dans l'ensemble de données (dans ce cas, 10).

Parce que ResNet est une architecture très profonde, lourde et lente à former, nous avons vérifié le modèle après chaque époque. Dans cette recette, nous avons obtenu le meilleur modèle à l'époque 38, qui a produit une précision de 72% sur l'ensemble de test, une performance respectable étant donné que CINIC-10 n'est pas un ensemble de données facile et que nous n'avons appliqué aucune augmentation de données ou transfert d'apprentissage.